## *Data Mining*

#### *Data Mining is the process of discovering patterns, relationships, and useful insights from large sets of data. It’s like finding valuable information hidden in a big pile of data*

In [ ]:
df = pd.read_parquet('../data/final/olist_full_data_Loaded.parquet', engine='pyarrow')
df

## *1.Clustering (Unsupervised Learning)*

### *K-Means*

##### *Step 1:Choose Relevant Columns for Clustering*
*Since K-Means works only with numerical values, we need to pick columns that are:Meaningful for identifying groups of similar orders or customers*
- *Numerical*



In [ ]:
features_to_cluster = ['price','freight_value','payment_installments','payment_value', 'review_score', 'product_weight_g', 'product_volume_cm3', 'product_density','profit_margin',
                        'freight_ratio','order_processing_time','recency_days','purchase_frequency','monetary_value']
df_cluster = df[features_to_cluster].copy()


#### *Step 2: Handling Missing Values*

In [ ]:
df_cluster.isnull().sum()
# Fill missing values with the median
df_cluster = df_cluster.fillna(df_cluster.median(numeric_only=True))